In [ ]:
import sys

sys.path.append("../")
import numpy as np
import matplotlib.pyplot as plt
import os
import json
import re
import cv2
import json
from pprint import pprint
import random
from sklearn.model_selection import StratifiedKFold
from monkey.data.data_utils import (
    open_json_file,
    write_json_file,
    extract_id,
)

Cell Detection 5-Fold Cross Validation Split.
Stratifiy using Centers

In [ ]:
wsi_dir = "/home/u1910100/Downloads/Monkey/images/pas-cpg"
wsi_names = os.listdir(wsi_dir)
wsi_names.sort()
# pprint(wsi_names)

split_labels = []
for name in wsi_names:
    centre = name[0]
    if centre == "A":
        split_labels.append(0)
    elif centre == "B":
        split_labels.append(1)
    elif centre == "C":
        split_labels.append(2)
    else:
        split_labels.append(3)

# pprint(split_labels)

In [ ]:
skf = StratifiedKFold(n_splits=5, random_state=None, shuffle=False)
skf.get_n_splits(wsi_names, split_labels)
for i, (train_index, test_index) in enumerate(
    skf.split(wsi_names, split_labels)
):
    pprint(f"Fold {i}:")
    pprint(f"Train: index={train_index}")
    pprint(f"Test:  index={test_index}")

In [ ]:
# Save WSI level split info
wsi_split_info = {}
for i, (train_index, test_index) in enumerate(
    skf.split(wsi_names, split_labels)
):
    # pprint(f"Fold {i}:")
    train_files = []
    test_files = []
    train_files = [wsi_names[i] for i in train_index]
    test_files = [wsi_names[i] for i in test_index]
    # pprint(f"Train files:={train_files}")
    # pprint(f"Test files={test_files}")

    wsi_split_info[f"Fold_{i+1}"] = {
        "train_files": train_files,
        "test_files": test_files,
    }

pprint(wsi_split_info)

save_path = (
    "/home/u1910100/Documents/Monkey/patches_256/wsi_level_split.json"
)
write_json_file(save_path, wsi_split_info)

In [ ]:
# Create patch level splits from wsi level splits

patches_stats_path = (
    "/home/u1910100/Documents/Monkey/patches_256/patch_stats.json"
)

patch_stats = open_json_file(patches_stats_path)

# pprint(patch_stats)

patch_names = list(patch_stats.keys())
patch_names.sort()
# pprint(patch_names)

wsi_name_patch_dict = {}

for name in patch_names:
    wsi_name = extract_id(name)
    wsi_name_patch_dict[name] = wsi_name

# pprint(wsi_name_patch_dict)
#     patch_ids.append(id)

# pprint(patch_ids)

patch_split_info = {}

for fold_data in wsi_split_info.items():
    fold_number = fold_data[0]
    train_wsis = fold_data[1]["train_files"]
    test_wsis = fold_data[1]["test_files"]

    train_wsi_ids = [extract_id(name) for name in train_wsis]
    test_wsi_ids = [extract_id(name) for name in test_wsis]

    # pprint(test_wsi_ids)
    train_patches = []
    test_patches = []

    for patch_item in wsi_name_patch_dict.items():
        patch_name = patch_item[0]
        wsi_id = patch_item[1]
        if wsi_id in test_wsi_ids:
            test_patches.append(patch_name)
        else:
            train_patches.append(patch_name)

    patch_split_info[fold_number] = {
        "train_files": train_patches,
        "test_files": test_patches,
    }

# pprint(patch_split_info)
save_path = "/home/u1910100/Documents/Monkey/patches_256/patch_level_split.json"
write_json_file(save_path, patch_split_info)

Classification 5-Fold CV-Split. Stratify using class labels

In [ ]:
%reset -f
import numpy as np
import matplotlib.pyplot as plt
import os
import json
import re
import cv2
import json
from pprint import pprint
import random
from sklearn.model_selection import StratifiedKFold
from monkey.data.data_utils import (
    open_json_file,
    write_json_file,
    extract_id,
    get_label_from_class_id,
)

In [ ]:
patches_dir = "/home/u1910100/Documents/Monkey/classification/patches"

patch_names = os.listdir(patches_dir)
patch_names.sort()
# pprint(patch_names)

split_labels = []
for name in patch_names:
    id = os.path.splitext(name)[0]
    label = get_label_from_class_id(id)
    if label == 0:
        split_labels.append(0)
    else:
        split_labels.append(1)

In [ ]:
skf = StratifiedKFold(n_splits=5, random_state=None, shuffle=False)
skf.get_n_splits(patch_names, split_labels)
for i, (train_index, test_index) in enumerate(
    skf.split(patch_names, split_labels)
):
    pprint(f"Fold {i}:")
    pprint(f"Train: index={train_index}")
    pprint(f"Test:  index={test_index}")

In [ ]:
# Save split info
patch_split_info = {}
for i, (train_index, test_index) in enumerate(
    skf.split(patch_names, split_labels)
):
    # pprint(f"Fold {i}:")
    train_files = []
    test_files = []
    train_files = [
        os.path.splitext(patch_names[i])[0] for i in train_index
    ]
    test_files = [
        os.path.splitext(patch_names[i])[0] for i in test_index
    ]
    # pprint(f"Train files:={train_files}")
    # pprint(f"Test files={test_files}")

    patch_split_info[f"Fold_{i+1}"] = {
        "train_files": train_files,
        "test_files": test_files,
    }

pprint(patch_split_info)

save_path = "/home/u1910100/Documents/Monkey/classification/patch_level_split.json"
write_json_file(save_path, patch_split_info)